In [1]:
!pip install psycopg2

In [15]:
!pip install python-dotenv

In [24]:
import psycopg2 # library for working with Postgres database
from dotenv import load_dotenv # library for working with environment variables
import os

In [25]:
load_dotenv()  # Load environment variables from .env file

True

In [9]:
# Read connection paramenters from environment variables (BEST PRACTICE!)
host = os.environ.get('DB_HOST')
dbname = os.environ.get('DB_NAME')
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASS')

In [47]:
# Other parameters
path = os.environ.get('FILE_PATH')
tbl_airport = 'airports'
tbl_flight = 'flights'
tbl_territories = 'territories'

In [15]:
print(f"host={host}") # Testing if env variables were successfully loaded

host=localhost


In [16]:
print(f"host={dbname}") # Testing if env variables were successfully loaded

host=raw_inventory


In [29]:
full_path = f'{path}/{tbl_airport}.csv' # Testing if env variables were successfully loaded
print(full_path)

C:/Users/angel/Desktop/DE PROJECTS/Professional-Portfolio/Building Data Model and Writing ETL Job/Data/airports.csv


In [49]:
# Start connection to Postgres Database 
# Using WITH (context manager) to ensure the connection is properly closed (BEST PRACTICE!), 
try:
    with psycopg2.connect(f"host={host} dbname={dbname} user={user} password={password}") as conn:
        # Create a cursor object
        with conn.cursor() as cursor:
            # Create a table if not exists
            # create_airport_table = "CREATE TABLE IF NOT EXISTS territories (airport_id INT, city VARCHAR, state VARCHAR, name VARCHAR)"
            # create_flights_table = "CREATE TABLE IF NOT EXISTS flights (DayofMonth INT, DayOfWeek INT, Carrier VARCHAR, OriginAirportID INT, DestAirportID INT, DepDelay INT, ArrDelay INT)"
            
            create_territory_table = f"CREATE TABLE IF NOT EXISTS {tbl_territories} (SalesTerritoryKey INT, Region VARCHAR, Country VARCHAR, Continent VARCHAR)"
            
            # cursor.execute(create_airport_table)
            # cursor.execute(create_flights_table)
            cursor.execute(create_territory_table)

            # Read airports CSV file and load data into airports table
            # with open(f'{path}/{tbl_airport}.csv', 'r') as file:
            #     next(file) #skip first row
            #     cursor.execute(f"TRUNCATE TABLE {tbl_airport}")
            #     cursor.copy_from(file, tbl_airport, sep=',' )

            # # Read flights CSV file and load data into flights table
            # with open(f'{path}/{tbl_flight}.csv', 'r') as file:
            #     next(file) #skip first row
            #     cursor.execute(f"TRUNCATE TABLE {tbl_flight}")
            #     cursor.copy_from(file, tbl_flight, sep=',' )
            
            # Read territories CSV file and load data into territories table
            with open(f'{path}/{tbl_territories}.csv', 'r') as file:
                next(file) #skip first row
                cursor.execute(f"TRUNCATE TABLE {tbl_territories}")
                cursor.copy_from(file, tbl_territories, sep=',' )

    conn.commit()
except psycopg2.OperationalError as e:
    print("Could not start a connection to the database")
    print(e)
except psycopg2.DatabaseError as e:
    print("An error occur while working with the database")
    print(e)

host=localhost
